In [1]:
from openai import OpenAI
from pydantic import BaseModel
import pyodbc
import json

# Set variables and output format for LLM

In [2]:
# LLM output format
class SqlOutput(BaseModel):
    sql_query: str

In [3]:
MODEL = "gpt-4o-2024-08-06"  # you could also use "gpt-4o-mini", but its code generation quality is poorer

In [4]:
# You could use below text to convert to SQL or replace it with your own
text_to_translate = "Provide a list of employees, their birthdays and age assuming it's 22th August 2024, \
including only employees hired between November-December 2007. Provide employee name, birthday and age"

# Get table representation

In [5]:
DRIVER = 'SQL Server'
SERVERNAME = <insert your servername>
DATABASE = 'AdventureWorks'  # change your database if applicable
CHOOSE_SCHEMA = True  # retrieve only schemas below (True) or get all SQL schemas (False)
if CHOOSE_SCHEMA:
    # Select schemas to retrieve, in order to provide less input to the prompt.
    # Start and end with double quotation mark, each specified
    # column should be surrounded with single quotation marks and each column should be
    # separated with a comma
    schemas = "'Sales', 'Person', 'HumanResources'"
descr_cols = 'TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, DATA_TYPE'

In [6]:
# Initialize SQL connection
con = pyodbc.connect(driver=DRIVER, server=SERVERNAME, database=DATABASE)

In [7]:
# Query SQL to get database representation
cur = con.cursor()
if CHOOSE_SCHEMA:
    db_cmd = f"SELECT {descr_cols} FROM INFORMATION_SCHEMA.COLUMNS where TABLE_SCHEMA in ({schemas})"
else:
    db_cmd = f"SELECT {descr_cols} FROM INFORMATION_SCHEMA.COLUMNS"
results = cur.execute(db_cmd)

In [8]:
'''
# Code for generating table representation for other SQL dialects. Not tested!

client = OpenAI()

#completion = client.chat.completions.create(
completion = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are an expert is converting human language to SQL"},
        {"role": "user", "content": f"""Create an SQL query to get a representation of all tables from a database.
It should output the following columns: 'TABLE_SCHEMA', 'TABLE_NAME', 'COLUMN_NAME' and 'DATA_TYPE'.            
Use Oracle syntax.
Keep in mind that SQL doesn't support new lines, so don't use `\n` in your output.
It's crucial that the generated code is compatible with SQL editors.
Provide only SQL code. It will be coppied to an SQL editor, so make sure that only necessary text is provided to not \
throw any errors in SQL.
"""
        }
    ],
    response_format=SqlOutput,
    temperature=0.0
)

db_cmd = json.loads(completion.choices[0].message.content)['sql_query']

print(db_cmd)

cur = con.cursor()
results = cur.execute(db_cmd)
'''

'\n# Code for generating table representation for other SQL dialects. Not tested!\n\nclient = OpenAI()\n\n#completion = client.chat.completions.create(\ncompletion = client.beta.chat.completions.parse(\n    model=MODEL,\n    messages=[\n        {"role": "system", "content": "You are an expert is converting human language to SQL"},\n        {"role": "user", "content": f"""Create an SQL query to get a representation of all tables from a database.\nIt should output the following columns: \'TABLE_SCHEMA\', \'TABLE_NAME\', \'COLUMN_NAME\' and \'DATA_TYPE\'.            \nUse Oracle syntax.\nKeep in mind that SQL doesn\'t support new lines, so don\'t use `\n` in your output.\nIt\'s crucial that the generated code is compatible with SQL editors.\nProvide only SQL code. It will be coppied to an SQL editor, so make sure that only necessary text is provided to not throw any errors in SQL.\n"""\n        }\n    ],\n    response_format=SqlOutput,\n    temperature=0.0\n)\n\ndb_cmd = json.loads(comple

In [9]:
# Transform results to a list with string values
table_descr = [str(row) for row in results]

In [10]:
# Number of columns to examine
len(table_descr)

443

In [11]:
# Number of characters
len(str(table_descr))

27140

In [12]:
# First 100 columns summary
table_descr[:100]

["('Person', 'Password', 'BusinessEntityID', 'int')",
 "('Person', 'Password', 'PasswordHash', 'varchar')",
 "('Person', 'Password', 'PasswordSalt', 'varchar')",
 "('Person', 'Password', 'rowguid', 'uniqueidentifier')",
 "('Person', 'Password', 'ModifiedDate', 'datetime')",
 "('Sales', 'SalesPersonQuotaHistory', 'BusinessEntityID', 'int')",
 "('Sales', 'SalesPersonQuotaHistory', 'QuotaDate', 'datetime')",
 "('Sales', 'SalesPersonQuotaHistory', 'SalesQuota', 'money')",
 "('Sales', 'SalesPersonQuotaHistory', 'rowguid', 'uniqueidentifier')",
 "('Sales', 'SalesPersonQuotaHistory', 'ModifiedDate', 'datetime')",
 "('Person', 'Person', 'BusinessEntityID', 'int')",
 "('Person', 'Person', 'PersonType', 'nchar')",
 "('Person', 'Person', 'NameStyle', 'bit')",
 "('Person', 'Person', 'Title', 'nvarchar')",
 "('Person', 'Person', 'FirstName', 'nvarchar')",
 "('Person', 'Person', 'MiddleName', 'nvarchar')",
 "('Person', 'Person', 'LastName', 'nvarchar')",
 "('Person', 'Person', 'Suffix', 'nvarchar')"

# Generate SQL code

In [13]:
client = OpenAI()

completion = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are an expert is converting human language to SQL"},
        {"role": "user", "content": f"""Generate an SQL query based on instructions from the following text.
Use {DRIVER} syntax.
Keep in mind that SQL doesn't support new lines, so using \n newline in your output is forbidden.
It's crucial that the generated code is compatible with SQL editors.
Text: {text_to_translate}.
Use the SQL table representation below:
Columns: {descr_cols}
Values: {table_descr}
"""
        }
    ],
    response_format=SqlOutput,
    temperature=0.0
)

output_query = json.loads(completion.choices[0].message.content)['sql_query']
print(output_query)

SELECT CONCAT(p.FirstName, ' ', p.LastName) AS EmployeeName, e.BirthDate, DATEDIFF(YEAR, e.BirthDate, '2024-08-22') AS Age FROM HumanResources.Employee e JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID WHERE e.HireDate BETWEEN '2007-11-01' AND '2007-12-31'


# Generate final SQL code with syntax check

In [14]:
client = OpenAI()

completion = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are an SQL quality code expert, specializing in correcting errors in code."},
        {"role": "user", "content": f"""Check if the below SQL query is a correct code implementation \
based on the instructions and schema description below it.
Make sure that {DRIVER} syntax was correctly implemented.
Make sure that the generated code is compatible with SQL editors.

Generate SQL query again, fixing errors if applicable or keeping the original code.
Keep in mind that SQL doesn't support new lines, so don't use `\n` in your output.
It's crucial that the generated code is compatible with SQL editors.
Generated code to correct: `{output_query}`
Instructions: {text_to_translate}.
Use the SQL table representation below:
Columns: {descr_cols}
Values: {table_descr}
"""
        }
    ],
    response_format=SqlOutput,
    temperature=0.0
)

final_query = json.loads(completion.choices[0].message.content)['sql_query']
print(final_query)

SELECT CONCAT(p.FirstName, ' ', p.LastName) AS EmployeeName, e.BirthDate, DATEDIFF(YEAR, e.BirthDate, '2024-08-22') AS Age FROM HumanResources.Employee e JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID WHERE e.HireDate BETWEEN '2007-11-01' AND '2007-12-31'
